# Video1: Overview - RAG From scratch

### organized_data klasöründen en basit haliyle cevap veriyor.

In [3]:
# Gerekli kütüphanelerin eklenmesi
import os
import getpass
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_community.document_loaders import DirectoryLoader, TextLoader

# .env dosyasını yükleyerek API anahtarlarını getir
load_dotenv() 

# API Anahtarları
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
#os.environ['LANGCHAIN_TRACING_V2'] = 'true'
#os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

# Model Tanımlaması
model = ChatOpenAI(model="gpt-4")
embedding = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

#### INDEXING ####

# Ana veri dizini: Tüm kategoriler altındaki dosyaları yükleme
base_directory = "rag_data/website/organized_data"

# DirectoryLoader ile belirtilen dizindeki tüm alt klasörlerden txt dosyalarını yükle
loader = DirectoryLoader(base_directory, glob="**/*.txt", loader_cls=TextLoader)

# Belgeleri yükle
docs = loader.load()

# Embedding işlemi: Text Splitter gerekmediği için belgeleri direkt olarak embed ediyoruz
vectorstore = Chroma.from_documents(documents=docs, 
                                    embedding=embedding)

# Retriever tanımlaması
retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt: Hub'dan prompt çekiliyor
custom_prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template(
        """
        You are a Telekom-Hilfe assistant for question-answering tasks, providing answers to Telekom customers or potential customers.
        Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
        Use three sentences maximum and keep the answer concise.
        Question: {question}
        Context: {context}
        Answer:
        """
    )
])

# Belgeleri formatlama fonksiyonu
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain tanımlaması
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_prompt
    | model
    | StrOutputParser()
)

# Soru: Zinciri çalıştırarak soruyu soruyoruz
result = rag_chain.invoke("Magenta tv hakkinda bilgi verir misin?")
print(result)

MagentaTV, Telekom tarafından sağlanan bir televizyon hizmetidir. İster evde ister yolda, TV-Receiver, MagentaTV One veya MagentaTV App ile keyifli bir televizyon deneyimi yaşayabilirsiniz. MagentaTV+ ile aksiyon, drama, özgün ve özel içerikler, uluslararası en iyi diziler ve filmler ile ARD & ZDF'nin en iyisi dahildir. 180'den fazla kanal, bunlardan 100'ü özel ve kamu hizmetine ait HD kalitesinde kanallardır. Ayrıca, MagentaTV One, MagentaTV Stick ve MagentaTV App ile kişisel hızlı başlangıç çubuğunu kullanabilirsiniz.


### web_data daki bütün txt dosyalarina bakarak soruya cevap veriyor, ve bilgi kaynagini veriyor.

In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
import asyncio

# .env dosyasını yükleyerek API anahtarlarını getir
load_dotenv()

# API Anahtarları
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

# Model Tanımlaması
model = ChatOpenAI(model="gpt-4", api_key=OPENAI_API_KEY)

#### INDEXING ####

# Ana veri dizini: Tüm kategoriler altındaki dosyaları yükleme
base_directory = "rag_data/website/organized_data"

# DirectoryLoader ile belirtilen dizindeki tüm alt klasörlerden txt dosyalarını yükle
loader = DirectoryLoader(base_directory, glob="**/*.txt", loader_cls=TextLoader)

# Belgeleri yükle
docs = loader.load()

# Embedding işlemi
embedding = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectorstore = Chroma.from_documents(documents=docs, embedding=embedding)

# Retriever tanımlaması
retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt: ChatPromptTemplate kullanılarak prompt hazırlanır
custom_prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template(
        """
        You are a Telekom-Hilfe assistant for question-answering tasks, providing answers to Telekom customers or potential customers.
        Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
        Use three sentences maximum and keep the answer concise.
        Question: {question}
        Context: {context}
        Answer:
        """
    )
])

# Belgeleri formatlama fonksiyonu
def format_docs(docs):
    unique_sources = set()  # Benzersiz kaynakları saklamak için bir set
    formatted_docs = []
    
    for doc in docs:
        source = doc.metadata.get("source")  # Kaynağı metadata'dan al
        if source and source not in unique_sources:
            unique_sources.add(source)
            formatted_docs.append(f"Source URL: {source}\n\n{doc.page_content}")
    
    return "\n\n".join(formatted_docs)

# Chain tanımlaması
rag_chain = (
    {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
    | custom_prompt
    | model
    | StrOutputParser()
)

# Sorgu için kullanılan belgeleri ve cevabı döndüren fonksiyon
async def retrieve_and_format_docs(question):
    # Belgeleri sorgu için al
    retrieved_docs = await retriever.ainvoke(question)
    
    # Belgelerin içeriğini formatla
    formatted_docs = format_docs(retrieved_docs)
    
    # Cevabı al
    try:
        answer = await rag_chain.invoke({"context": formatted_docs, "question": question})
    except TypeError:
        # Eğer TypeError alınırsa, invoke çağrısının senkron olduğu anlamına gelebilir
        answer = rag_chain.invoke({"context": formatted_docs, "question": question})
    
    return answer, formatted_docs

# Jupyter Notebook gibi bir ortamda, mevcut olay döngüsünü kullanarak asenkron işlevleri çağırma
async def main():
    question = "TV izlemek icin hangi ürünlerinizi onerebilirsin?"
    answer, source_docs = await retrieve_and_format_docs(question)
    print("Answer:\n", answer)
    print("\nSources:\n", source_docs)

# Mevcut olay döngüsünü kullanarak asenkron işlevleri çalıştırma
await main()

Answer:
 TV izlemek için MagentaTV+ ve WOW paketlerini önerebilirim. MagentaTV+, HD kalitesinde 180'den fazla kanal sunar ve kişiselleştirilebilir bir hızlı başlatma çubuğu ile favori uygulamalarınıza hızlıca erişim sağlar. WOW paketi ile Sky'dan en son dizileri, en son hit filmleri ve en iyi canlı sporları izleyebilirsiniz. Ayrıca MagentaTV, Netflix, Prime Video, Disney+ ve daha birçoklarını içeren 100'den fazla streaming servisine ve partner uygulamasına erişim sağlar.

Sources:
 Source URL: rag_data/website/organized_data/Others/https_www_telekom_de_magenta_tv_tarife_und_optionen_magenta_tv_mit_app.txt

Source URL: https://www.telekom.de/magenta-tv/tarife-und-optionen/magenta-tv-mit-app

1. Question: Welches TV Produkt passt am besten zu dir?
   Answer: MagentaTV+
Von Action bis Drama. Von Originals & Exclusivesbis zu internationalen Top-Serien & Filmen sowie dem Besten von ARD & ZDF. Alles inklusive. Bei MagentaTV ist das + immer dabei.Zu MagentaTV+
HD-Sender
Beste Fernseh-Unterhal

### kaynak dosya adi, kaynak url, cos similarity ve kaynak soru cevaplarla veriyor ciktiyi..

In [12]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
import numpy as np
import asyncio

# .env dosyasını yükleyerek API anahtarlarını getir
load_dotenv()

# API Anahtarları
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

# Model Tanımlaması
model = ChatOpenAI(model="gpt-4", api_key=OPENAI_API_KEY)

#### INDEXING ####

# Ana veri dizini: Tüm kategoriler altındaki dosyaları yükleme
base_directory = "rag_data/website/organized_data"

# DirectoryLoader ile belirtilen dizindeki tüm alt klasörlerden txt dosyalarını yükle
loader = DirectoryLoader(base_directory, glob="**/*.txt", loader_cls=TextLoader)

# Belgeleri yükle
docs = loader.load()

# Embedding işlemi
embedding = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectorstore = Chroma.from_documents(documents=docs, embedding=embedding)

# Retriever tanımlaması
retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt: ChatPromptTemplate kullanılarak prompt hazırlanır
custom_prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template(
        """
        You are a Telekom-Hilfe assistant for question-answering tasks, providing answers to Telekom customers or potential customers.
        Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
        Use three sentences maximum and keep the answer concise.
        Question: {question}
        Context: {context}
        Answer:
        """
    )
])

# Cosine Similarity hesaplama fonksiyonu
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

# Belgeleri formatlama fonksiyonu
def format_docs(docs, query_embedding):
    unique_sources = set()  # Benzersiz kaynakları saklamak için bir set
    formatted_docs = []
    
    for doc in docs:
        source = doc.metadata.get("source")  # Kaynağı metadata'dan al
        if source and source not in unique_sources:
            unique_sources.add(source)
            document_embedding = embedding.embed_query(doc.page_content)  # Embedding hesapla
            similarity = cosine_similarity(query_embedding, document_embedding)  # Cosine similarity hesapla
            content = doc.page_content.strip() or "Bu belge içeriği boş."  # Belge içeriği
            formatted_docs.append(
                f"Source document: {source}\n\nCosine Similarity: {similarity:.4f}\n\n{content}"
            )
    
    return "\n\n".join(formatted_docs)

# Chain tanımlaması
rag_chain = (
    {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
    | custom_prompt
    | model
    | StrOutputParser()
)

# Sorgu için kullanılan belgeleri ve cevabı döndüren fonksiyon
async def retrieve_and_format_docs(question):
    # Belgeleri sorgu için al
    query_embedding = embedding.embed_query(question)  # Sorgu için embedding al
    retrieved_docs = await retriever.ainvoke(question)
    
    # Belgelerin içeriğini formatla
    formatted_docs = format_docs(retrieved_docs, query_embedding)
    
    # Cevabı al
    try:
        answer = await rag_chain.invoke({"context": formatted_docs, "question": question})
    except TypeError:
        # Eğer TypeError alınırsa, invoke çağrısının senkron olduğu anlamına gelebilir
        answer = rag_chain.invoke({"context": formatted_docs, "question": question})
    
    return answer, formatted_docs

# Jupyter Notebook gibi bir ortamda, mevcut olay döngüsünü kullanarak asenkron işlevleri çağırma
async def main():
    question = "Bir mobil hat almak istiyorum, hangi opsiyonlar mevcut?"
    answer, source_docs = await retrieve_and_format_docs(question)
    print("Answer:", answer)
    print("\nSources:")
    print(source_docs)

# Mevcut olay döngüsünü kullanarak asenkron işlevleri çalıştırma
await main()

Answer: Telekom'da çeşitli "Travel & Mobil" yurtdışı seçeneklerini mevcuttur. Bu opsiyonları, müşteri merkezi veya MeinMagenta uygulaması üzerinden kendiniz satın alabilirsiniz. Hangi opsiyonu seçeceğinize karar vermek için, müşteri merkezinde veya uygulamada "Zubuchoptionen hinzufügen" (Ek opsiyonlar ekle) butonuna tıklayarak tüm seçenekleri görebilirsiniz.

Sources:
Source document: rag_data/website/organized_data/Mobilfunk/https_www_telekom_de_hilfe_mobilfunk_tarife_optionen_ausland_roaming_travel_mobil_buchen.txt

Cosine Similarity: 0.7902

Source URL: https://www.telekom.de/hilfe/mobilfunk/tarife-optionen/ausland-roaming/travel-mobil-buchen
Telekom > Hilfe & Service > Mobilfunk > Tarife & Optionen > Ausland & Roaming > Buchung > Travel > Mobil

1. Question: Wie buche ich eine Travel Mobil Option?
   Answer: Sie können die unterschiedlichen Auslandsoptionen Travel & Mobil im Kundencenter und der MeinMagenta App selbst buchen.
KundencenterKlicken Sie imKundencenterauf "Zum Vertrag" 

## Video2: Indexing

In [ ]:
# Documents
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

In [ ]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

In [ ]:
from langchain_openai import OpenAIEmbeddings
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result)
# sorunun embedding olarak ne kadar uzun oldugunu yazdiriyor, 1536...

In [ ]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

In [ ]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

In [ ]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

In [ ]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

## Video3: Retrieval

In [ ]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())


retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [ ]:
docs = retriever.invoke("What is Task Decomposition?")
len(docs)
print(docs)

## Video4: Generation

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

In [ ]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

In [ ]:
# Chain
chain = prompt | llm

In [ ]:
# Run
chain.invoke({"context":docs,"question":"What is Task Decomposition?"})

In [ ]:
from langchain import hub
prompt_hub_rag = hub.pull("rlm/rag-prompt")

In [ ]:
prompt_hub_rag

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_hub_rag
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")